Description : This script loads and transforms data from the staging table 
              (VESSOPS_D.L00_STG.GCI_PRICING_CHECK_AK_1) into the RDV satellite table 
              (VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRICING_RG_PASS). 
              It applies business rules such as handling null ship codes for specific cruise lines, 
              enriching data with reference ship details (IMO, ship code), and generating derived 
              identifiers (SAIL_ID, MD5_VOYAGE_HASH_KEY, HASH_DIFF). 
              Metadata columns (LDTS, RCSR, LAST_MODIFIED_BY) are also populated.

Author      : Rakesh J (Mu-Sigma)
Date        : 2025-08-26

Updates     : Nil
Date        : Nil

## Staging to RDV

In [ ]:
INSERT INTO VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRICING_RG_PASS
SELECT
    -- Columns from the source staging table
    stg.RATEINQUIRYDATE,
    stg.SOURCEMARKET,
    stg.CRUISELINECODE,
    -- 1. Handle NULL shipcodes for specific cruise lines
    CASE
        WHEN stg.shipcode IS NULL AND stg.cruiselinecode IN ('HAL', 'SIL') THEN 'NA'
        ELSE stg.shipcode
    END AS SHIPCODE,
    stg.CRUISELINENAME,
    stg.SOURCESHIPNAME,
    stg.WEBSITE,
    stg.CACHEPAGELINK,
    stg.CABINTYPE,
    stg.CATEGORYCODE,
    stg.SOURCECURRENCY,
    stg.FAREPASSENGERTYPE,
    stg.ORIGCURRENCY,
    stg.RATEPPUSD,
    stg.TAXESANDFEES,
    stg.TAXESANDFEESUSD,
    stg.CONVERTIONRATE,
    stg.CRUISETYPE,
    stg.CRUISEDURATION,
    stg.DISEMBARKDATE,
    stg.SHOPFREQUENCY,
    stg.EMBARKATIONDATE,
    stg.SOURCEITINERARYNAME,
    stg.EMBARKATIONPORT,
    stg.EMBARKATIONPORTUN_LOCODE,
    stg.DISEMBARKATIONPORT,
    stg.DISEMBARKATIONPORTUN_LOCODE,
    stg.ITINERARYPORTCONTENT,
    stg.RATETYPE,
    stg.TAXINCLUSIVERATEPP,
    stg.TAXEXCLUSIVERATEPP,
    stg.AVAILABLEOFFER,
    stg.LASTSHOPDATE,
    stg.LASTSHOPPEDSUBCATEGORY,
    stg.AVAILABILITY,
    stg.OBSTRUCTEDYN,
    stg.FILEPATH,
    stg.GRATUITYYN,
    stg.NCL_CABINTYPE,
    stg.RSS_CABINTYPE,
    stg.OCI_CABINTYPE,

    -- 2. Generate new columns on the fly
    -- Get VESSEL_IMO and NCLH_SHIP_CODE from the reference table
    ref.VESSEL_IMO_NO AS VESSEL_IMO,
    ref.ship_code_3l AS NCLH_SHIP_CODE,

    -- Calculate SAIL_ID
    CONCAT(
        ref.ship_code_3l,
        TO_CHAR(stg.EMBARKATIONDATE, 'YYMMDD')
    ) AS SAIL_ID,

    -- Calculate MD5_VOYAGE_HASH_KEY
    MD5(
        CONCAT(
            ref.ship_code_3l,
            TO_CHAR(stg.EMBARKATIONDATE, 'YYMMDD')
        )
    ) AS MD5_VOYAGE_HASH_KEY,

    -- Calculate HASH_DIFF
    MD5(
        CONCAT_WS(
            '|',
            COALESCE(TO_VARCHAR(stg.RATEINQUIRYDATE), ''),
            COALESCE(stg.SOURCEMARKET, ''),
            COALESCE(stg.CRUISELINECODE, ''),
            COALESCE(CASE WHEN stg.shipcode IS NULL AND stg.cruiselinecode IN ('HAL', 'SIL') THEN 'NA' ELSE stg.shipcode END, ''),
            COALESCE(stg.CRUISELINENAME, ''),
            COALESCE(stg.SOURCESHIPNAME, ''),
            COALESCE(stg.WEBSITE, ''),
            COALESCE(stg.CACHEPAGELINK, ''),
            COALESCE(stg.CABINTYPE, ''),
            COALESCE(stg.CATEGORYCODE, ''),
            COALESCE(stg.SOURCECURRENCY, ''),
            COALESCE(stg.FAREPASSENGERTYPE, ''),
            COALESCE(stg.ORIGCURRENCY, ''),
            COALESCE(TO_VARCHAR(stg.RATEPPUSD), ''),
            COALESCE(TO_VARCHAR(stg.TAXESANDFEES), ''),
            COALESCE(TO_VARCHAR(stg.TAXESANDFEESUSD), ''),
            COALESCE(TO_VARCHAR(stg.CONVERTIONRATE), ''),
            COALESCE(stg.CRUISETYPE, ''),
            COALESCE(TO_VARCHAR(stg.CRUISEDURATION), ''),
            COALESCE(TO_VARCHAR(stg.DISEMBARKDATE), ''),
            COALESCE(stg.SHOPFREQUENCY, ''),
            COALESCE(TO_VARCHAR(stg.EMBARKATIONDATE), ''),
            COALESCE(stg.SOURCEITINERARYNAME, ''),
            COALESCE(stg.EMBARKATIONPORT, ''),
            COALESCE(stg.EMBARKATIONPORTUN_LOCODE, ''),
            COALESCE(stg.DISEMBARKATIONPORT, ''),
            COALESCE(stg.DISEMBARKATIONPORTUN_LOCODE, ''),
            COALESCE(stg.ITINERARYPORTCONTENT, ''),
            COALESCE(stg.RATETYPE, ''),
            COALESCE(TO_VARCHAR(stg.TAXINCLUSIVERATEPP), ''),
            COALESCE(TO_VARCHAR(stg.TAXEXCLUSIVERATEPP), ''),
            COALESCE(stg.AVAILABLEOFFER, ''),
            COALESCE(TO_VARCHAR(stg.LASTSHOPDATE), ''),
            COALESCE(stg.LASTSHOPPEDSUBCATEGORY, ''),
            COALESCE(stg.AVAILABILITY, ''),
            COALESCE(stg.OBSTRUCTEDYN, ''),
            COALESCE(stg.FILEPATH, ''),
            COALESCE(stg.GRATUITYYN, ''),
            COALESCE(stg.NCL_CABINTYPE, ''),
            COALESCE(stg.RSS_CABINTYPE, ''),
            COALESCE(stg.OCI_CABINTYPE, '')
        )
    ) AS HASH_DIFF,

    -- Add metadata columns
    CURRENT_TIMESTAMP() AS LDTS,
    'RATEGAIN' AS RCSR,
    CURRENT_USER() AS LAST_MODIFIED_BY

FROM
    VESSOPS_D.L00_STG.GCI_PRICING_CHECK_AK_1 AS stg
LEFT JOIN
    VESSOPS_D.L00_STG.gci_raw_ref_ship_data AS ref
ON
    CONCAT(stg.CRUISELINECODE, stg.SHIPCODE) = ref."5L_SHIP_CODE";



#### Need to change the 'p_delivery_date'

In [ ]:
-- This script consolidates the logic into a single, optimized INSERT statement

SET p_delivery_date = '2025-08-22';



INSERT INTO VESSOPS_D.L10_RDV.GCI_NCH_INTER_REF_PRICING_DATA
WITH
-- CTE 1: Prepare the main pricing data (equivalent to t1)
pricing_data AS (
    SELECT DISTINCT
        pric.*,
        UPPER(TRIM(bs.MP_CruiseLineCode)) AS MP_CruiseLineCode,
        UPPER(TRIM(bs.MP_shipcode)) AS MP_shipcode,
        UPPER(TRIM(dt.Tb_Unv_Port_Code)) AS embarkport_locode,
        UPPER(TRIM(dtp.Tb_Unv_Port_Code)) AS disembarkport_locode
        -- Note: Embarkport and Disembarkport names are not used in the final query, so they are omitted here.
    FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRICING_RG_PASS AS pric
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_SHIP_DATA AS bs
        ON UPPER(TRIM(pric.cruiselinecode)) = UPPER(TRIM(bs.cruiselinecode))
        AND UPPER(TRIM(pric.shipcode)) = UPPER(TRIM(bs.shipcode))
        AND UPPER(TRIM(pric.SourceShipName)) = UPPER(TRIM(bs.SourceShipName))
        AND UPPER(TRIM(pric.CruiseLineName)) = UPPER(TRIM(bs.CruiseLineName))
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA AS dt
        ON UPPER(TRIM(pric.embarkationportun_locode)) = UPPER(TRIM(dt.Dtl_PortUN_Locode))
        AND UPPER(TRIM(pric.embarkationport)) = UPPER(TRIM(dt.Dtl_Port_City_Nam))
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA AS dtp
        ON UPPER(TRIM(pric.DisembarkationPortUN_LOCODE)) = UPPER(TRIM(dtp.Dtl_PortUN_Locode))
        AND UPPER(TRIM(pric.DisembarkationPort)) = UPPER(TRIM(dtp.Dtl_Port_City_Nam))
    -- Apply the filter as early as possible
    WHERE SUBSTRING(pric.Filepath, 42, 10) = $p_delivery_date
),

-- CTE 2: Prepare the voyage level data (equivalent to t2)
voy_level_data AS (
    SELECT DISTINCT
        UPPER(TRIM(hmp.mp_cruiselinecode)) AS cruiselinecode,
        UPPER(TRIM(hmp.mp_shipcode)) AS shipcode,
        hp.EmbarkationDate,
        hp.DisembarkationDate,
        hp.RateInquiryDate,
        hp.FilePath,
        UPPER(TRIM(tp.Tb_Unv_Port_Code)) AS EmbarkationPortUN_LOCODE,
        UPPER(TRIM(tp1.Tb_Unv_Port_Code)) AS DisembarkationPortUN_LOCODE,
        hp.CruiseDuration,
        hp.CachePageLink,
        'Q' AS ShopFrequency,
        'NA' AS Gratuity
    FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRODUCT_VOY_LEVEL_RG_PASS AS hp
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA AS tp
        ON UPPER(TRIM(hp.EmbarkationPort)) = UPPER(TRIM(tp.Dtl_Port_City_Nam))
        AND UPPER(TRIM(hp.EmbarkationPortUN_LOCODE)) = UPPER(TRIM(tp.Dtl_PortUN_Locode))
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA AS tp1
        ON UPPER(TRIM(hp.DisEmbarkationPort)) = UPPER(TRIM(tp1.Dtl_Port_City_Nam))
        AND UPPER(TRIM(hp.DisEmbarkationPortUN_LOCODE)) = UPPER(TRIM(tp1.Dtl_PortUN_Locode))
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_SHIP_DATA AS hmp
        ON UPPER(TRIM(hp.cruiselinecode)) = UPPER(TRIM(hmp.cruiselinecode))
        AND UPPER(TRIM(hp.shipcode)) = UPPER(TRIM(hmp.shipcode))
        AND UPPER(TRIM(hp.SourceShipName)) = UPPER(TRIM(hmp.SourceShipName))
        AND UPPER(TRIM(hp.CruiseLineName)) = UPPER(TRIM(hmp.CruiseLineName))
    -- Assuming the same date filter should apply here for consistency
   -- WHERE SUBSTRING(hp.Filepath, 37, 10) = $p_delivery_date
),

-- CTE 3: Prepare the voyage day level data (equivalent to t3)
voy_day_level_data AS (
    SELECT DISTINCT
        UPPER(TRIM(dmp.mp_cruiselinecode)) AS cruiselinecode,
        UPPER(TRIM(dmp.mp_shipcode)) AS shipcode,
        dp.EmbarkationDate,
        dp.DisembarkationDate,
        dp.RateInquiryDate,
        dp.FilePath,
        UPPER(TRIM(dtp.Tb_Unv_Port_Code)) AS EmbarkationPortUN_LOCODE,
        UPPER(TRIM(dtp1.Tb_Unv_Port_Code)) AS DisembarkationPortUN_LOCODE,
        dp.CruiseDuration,
        dp.CachePageLink,
        'Q' AS ShopFrequency,
        'NA' AS Gratuity
    FROM VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_PRODUCT_VOY_DAY_LEVEL_RG_PASS AS dp
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA AS dtp
        ON UPPER(TRIM(dp.EmbarkationPort)) = UPPER(TRIM(dtp.Dtl_Port_City_Nam))
        AND UPPER(TRIM(dp.EmbarkationPortUN_LOCODE)) = UPPER(TRIM(dtp.Dtl_PortUN_Locode))
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_PORT_DATA AS dtp1
        ON UPPER(TRIM(dp.disEmbarkationPort)) = UPPER(TRIM(dtp1.Dtl_Port_City_Nam))
        AND UPPER(TRIM(dp.disEmbarkationPortUN_LOCODE)) = UPPER(TRIM(dtp1.Dtl_PortUN_Locode))
    LEFT JOIN VESSOPS_D.L10_RDV.SAT_GCI_NCH_RAW_REF_MERGED_SHIP_DATA AS dmp
        ON UPPER(TRIM(dp.cruiselinecode)) = UPPER(TRIM(dmp.cruiselinecode))
        AND UPPER(TRIM(dp.shipcode)) = UPPER(TRIM(dmp.shipcode))
        AND UPPER(TRIM(dp.SourceShipName)) = UPPER(TRIM(dmp.SourceShipName))
        AND UPPER(TRIM(dp.CruiseLineName)) = UPPER(TRIM(dmp.CruiseLineName))
    -- Assuming the same date filter should apply here for consistency
    --WHERE SUBSTRING(dp.Filepath, 27, 10) = $p_delivery_date
),

-- CTE 4: Combine the data sources and calculate final columns
base AS (
    SELECT DISTINCT
        -- Canonical identity
        COALESCE(m.mp_cruiselinecode, k.cruiselinecode, g.cruiselinecode) AS CRUISELINECODE,
        COALESCE(m.mp_ShipCode, k.ShipCode, g.ShipCode) AS SHIPCODE,
        CAST(COALESCE(m.EmbarkationDate, k.EmbarkationDate, g.EmbarkationDate) AS TIMESTAMP_NTZ) AS EmbarkationDate,
        COALESCE(m.embarkport_locode, k.EmbarkationPortUN_LOCODE, g.EmbarkationPortUN_LOCODE) AS EmbarkationPortUN_LOCODE,

          -- Week/day
        CASE 
          WHEN DAYOFWEEK(COALESCE(m.EmbarkationDate, k.EmbarkationDate, g.EmbarkationDate)) = 7 THEN 1
          ELSE DAYOFWEEK(COALESCE(m.EmbarkationDate, k.EmbarkationDate, g.EmbarkationDate)) + 1
        END AS Day_of_Week,
        WEEKOFYEAR(COALESCE(m.EmbarkationDate, k.EmbarkationDate, g.EmbarkationDate)) AS Week_of_Year,
        CONCAT(
                COALESCE(m.mp_CruiseLineCode, k.CruiseLineCode, g.CruiseLineCode), '-',
                COALESCE(m.mp_ShipCode,       k.ShipCode,       g.ShipCode),       '-',
                CAST(COALESCE(m.EmbarkationDate, k.EmbarkationDate, g.EmbarkationDate) AS DATE), '-',
                COALESCE(m.embarkport_locode, k.EmbarkationPortUN_LOCODE, g.EmbarkationPortUN_LOCODE), '-',
                COALESCE(m.disembarkport_locode, k.DisembarkationPortUN_LOCODE, g.DisembarkationPortUN_LOCODE), '-',
                COALESCE(m.CruiseDuration, k.CruiseDuration, g.CruiseDuration)
            )  AS NCLH_UNI_VOYAGE_CODE,
        CASE 
          WHEN COALESCE(m.mp_CruiseLineCode, k.CruiseLineCode, g.CruiseLineCode) = 'NCL' THEN
            CONCAT(
                  COALESCE(m.mp_CruiseLineCode, k.CruiseLineCode, g.CruiseLineCode), '-',
                  COALESCE(m.mp_ShipCode,       k.ShipCode,       g.ShipCode),       '-',
                  CAST(COALESCE(m.EmbarkationDate, k.EmbarkationDate, g.EmbarkationDate) AS DATE), '-',
                  COALESCE(m.CruiseDuration, k.CruiseDuration, g.CruiseDuration)
                )
        END AS NCL_UNI_VOYAGE_CODE,
                -- Dates/ports
        COALESCE(m.DisEmbarkDate, k.DisEmbarkationDate, g.DisEmbarkationDate) AS DisEmbarkationDate,
        COALESCE(m.disembarkport_locode, k.DisembarkationPortUN_LOCODE, g.DisembarkationPortUN_LOCODE) AS DisembarkationPortUN_LOCODE,
        -- COALESCE(m.CruiseDuration, k.CruiseDuration, g.CruiseDuration) AS CruiseDuration,

        -- Pricing & meta
        m.TaxExclusiveRatePP,
        m.CabinType,
        CAST(COALESCE(m.RateInquiryDate, k.RateInquiryDate, g.RateInquiryDate) AS TIMESTAMP_NTZ) AS RateInquiryDate,
        COALESCE(m.CachePageLink, k.CachePageLink, g.CachePageLink) AS CachePageLink,
        COALESCE(m.ShopFrequency, k.ShopFrequency, g.ShopFrequency) AS ShopFrequency,
        COALESCE(m.GRATUITY_Y_N, k.Gratuity, g.Gratuity) AS "Gratuity (Y/N)", 
        m.LastShoppedSubCategory,
        m.RatePPUSD,
        m.TaxesandFees,
        m.TaxesandFeesUSD,
        m.RateType,
        m.TaxInclusiveRatePP,
        m.AvailableOffer,
        m.Availability,
        m.ConvertionRate,
        m.SOURCEMARKET,
        m.SourceCurrency,
        m.OrigCurrency,
        m.OBSTRUCTED_Y_N, 
        m.categorycode,
        m.FARE_PASSENGERTYPE, 
            -- File lineage (using the single coalesced filepath)
          COALESCE(SUBSTRING(m.filepath, 53, 16), 
          SUBSTRING(k.filepath, 48, 15), 
          SUBSTRING(g.filepath, 38, 15)
        ) AS File_NM,
        COALESCE(
          SUBSTRING(m.filepath, 42, 10), 
          SUBSTRING(k.filepath, 37, 10), 
          SUBSTRING(g.filepath, 27, 10)
        ) AS delivery_date,

 
        m.NCL_CABINTYPE,
        m.RSS_CABINTYPE,
        m.OCI_CABINTYPE,
        m.VESSEL_IMO,
        m.NCLH_SHIP_CODE,
        m.SAIL_ID,
        m.MD5_VOYAGE_HASH_KEY

    FROM pricing_data AS m
    LEFT JOIN voy_level_data AS k
        ON UPPER(TRIM(m.mp_cruiselinecode)) = k.CruiseLineCode -- Already cleaned in CTE
        AND UPPER(TRIM(m.mp_ShipCode)) = k.ShipCode
        AND TO_DATE(m.RateInquiryDate) = TO_DATE(k.RateInquiryDate)
    LEFT JOIN voy_day_level_data AS g
        ON UPPER(TRIM(m.mp_cruiselinecode)) = g.CruiseLineCode -- Already cleaned in CTE
        AND UPPER(TRIM(m.mp_ShipCode)) = g.ShipCode
        AND TO_DATE(m.RateInquiryDate) = TO_DATE(g.RateInquiryDate)

        WHERE COALESCE(SUBSTRING(m.filepath, 42, 10),
                        SUBSTRING(k.filepath, 37, 10),
                        SUBSTRING(g.filepath, 27, 10)
                        ) = $p_delivery_date
)

-- Final SELECT to calculate derived columns and the HASH_DIFF
SELECT
    b.*,
    -- Metadata
    CURRENT_TIMESTAMP AS LDTS,
    'Pricing Pass,Voy Level RG Pass,Voy Day Level RG Pass,Merged Port Data,Merged Ship Data' AS RCSR,
    CURRENT_USER AS LAST_MODIFIED_BY,

    -- Deterministic HASH_DIFF over business columns
    MD5(
        CONCAT_WS('|',
            -- identity & voyage
            COALESCE(b.CRUISELINECODE, ''),
            COALESCE(b.SHIPCODE, ''),
            COALESCE(TO_VARCHAR(b.EmbarkationDate, 'YYYY-MM-DD"T"HH24:MI:SS.FF9'), ''),
            COALESCE(b.EmbarkationPortUN_LOCODE, ''),
            COALESCE(TO_VARCHAR(Day_of_Week), ''),
            COALESCE(TO_VARCHAR(Week_of_Year), ''),
            COALESCE(NCLH_UNI_VOYAGE_CODE, ''),
            COALESCE(NCL_UNI_VOYAGE_CODE, ''),
            -- dates/ports
            COALESCE(TO_VARCHAR(b.DisEmbarkationDate, 'YYYY-MM-DD"T"HH24:MI:SS.FF9'), ''),
            COALESCE(b.DisembarkationPortUN_LOCODE, ''),
            -- pricing & attributes
            COALESCE(TO_VARCHAR(b.TaxExclusiveRatePP), ''),
            COALESCE(b.CabinType, ''),
            COALESCE(TO_VARCHAR(b.RateInquiryDate, 'YYYY-MM-DD"T"HH24:MI:SS.FF9'), ''),
            COALESCE(b.CachePageLink, ''),
            COALESCE(b.ShopFrequency, ''),
            COALESCE(b."Gratuity (Y/N)", ''),
            COALESCE(b.LastShoppedSubCategory, ''),
            COALESCE(TO_VARCHAR(b.RatePPUSD), ''),
            COALESCE(TO_VARCHAR(b.TaxesandFees), ''),
            COALESCE(TO_VARCHAR(b.TaxesandFeesUSD), ''),
            COALESCE(b.RateType, ''),
            COALESCE(TO_VARCHAR(b.TaxInclusiveRatePP), ''),
            COALESCE(b.AvailableOffer, ''),
            COALESCE(b.Availability, ''),
            COALESCE(TO_VARCHAR(b.ConvertionRate), ''),
            COALESCE(b.SOURCEMARKET, ''),
            COALESCE(b.SourceCurrency, ''),
            COALESCE(b.OrigCurrency, ''),
            COALESCE(b.OBSTRUCTED_Y_N, ''),
            COALESCE(b.categorycode, ''),
            COALESCE(b.FARE_PASSENGERTYPE, ''),
            -- lineage & extras
            COALESCE(File_NM, ''),
            COALESCE(delivery_date, ''),
            COALESCE(b.NCL_CABINTYPE, ''),
            COALESCE(b.RSS_CABINTYPE, ''),
            COALESCE(b.OCI_CABINTYPE, ''),
            COALESCE(TO_VARCHAR(b.VESSEL_IMO), ''),
            COALESCE(b.NCLH_SHIP_CODE, ''),
            COALESCE(TO_VARCHAR(b.SAIL_ID), '')
        )
    ) AS HASH_DIFF
FROM base AS b;
